In [1]:
# Dependencies
import json
import modules.dataset.entities as en
import modules.dataset.tweets as tw
import matplotlib.pyplot as plt

%matplotlib inline

# Constants
years = [2018, 2019]

# Dataset creation

Once words and hashtags are extracted, they are stored in Pandas DataFrames with the following attributes:

1. <code> id </code> : Reference to the tweet ID
2. <code> index </code> : Position of the word in the sentence (tweet) - useful to delete self loops (WORDS ONLY)
3. <code> text </code> : Text
4. <code> pos </code> : Part Of Speech tag (WORDS ONLY)
5. <code> conf </code> : Confidence associated to the POS tag (WORDS ONLY)

In [8]:
# Initialize Tweets class
tweets = tw.Tweets()
# Fill tweets dataframe
tweets.from_json('data/db/tweets_climatechange.json')
# Rename columns
tweets.df = tweets.df.rename(columns = {'id': 'tweet_id',
                                      'created_at': 'tweet_date',
                                      'text': 'tweet_text'})
tweets.df.head()

,tweet_date,tweet_id,tweet_text,hashtags,retweet_count,favorite_count,original_retweet_count,original_favorite
0,2018-04-23 02:23:27,988241915412873216,#GlobalWarming is one aspect of #ClimateChange...,"[globalwarming, climatechange, earthday]",0,0,40.0,57.0
1,2018-04-23 02:23:35,988241946073321472,We are still breaking the wrong records! Heat ...,"[climatechange, climateaction]",0,0,41.0,31.0
2,2018-04-23 02:24:03,988242065199943680,We responded to questions about the #Maskbook ...,"[maskbook, airpollution, climatechange, artofc...",0,0,11.0,12.0
3,2018-04-23 02:24:11,988242096812380160,"On this #EarthDay, we celebrate our members an...","[earthday, climateaction, climatechange, globa...",0,0,17.0,47.0
4,2018-04-23 02:24:11,988242096812380160,"On this #EarthDay, we celebrate our members an...","[earthday, climateaction, climatechange, globa...",0,0,17.0,47.0


In [9]:
# Initialize substitution dictionary
subs = {}
# Load hahstag substitutions
with open('data/hashtag_subs.json', 'r') as file:
    subs = {**subs, **json.load(file)}
# Load contact forms substitutions
with open('data/contract_forms.json', 'r') as file:
    subs = {**subs, **json.load(file)}

In [10]:
# Get hashtags and words datasets
hashtags, words = tweets.get_entities(subs=subs)
hashtags.df.head()

,tweet_id,entity_index,entity_text,entity_tag,entity_conf
0,988241915412873216,0,#GlobalWarming,#,0.7009
5,988241915412873216,5,#ClimateChange,^,0.8085
22,988241915412873216,22,#EarthDay,#,0.9694
46,988241946073321472,23,#climatechange,#,0.9188
47,988241946073321472,24,#climateaction,#,0.9933


In [11]:
words.df.head()

,tweet_id,entity_index,entity_text,entity_tag,entity_conf
0,988241915412873216,0,is,V,0.9904
1,988241915412873216,1,one,$,0.7817
2,988241915412873216,2,aspect,N,0.9988
3,988241915412873216,3,of,P,0.9966
4,988241915412873216,4,#ClimateChange,^,0.8085


In [13]:
# Save hashtags data
hashtags.to_json('data/db/hashtags.json')
#words.to_json('data/db/words.json')

## Words filtering

In [ ]:
# Filter on POS tag



In [ ]:
# Show confidence distribution

In [ ]:
# Filter on confidence

## Words cleaning